# Necessary imports for recipes generation

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, GPT2TokenizerFast
from torch.utils.data import Dataset, DataLoader
from recipes_generator.data.recipe_dataset import RecipeDataset, load_preprocess_raw_json_data, load_preprocess_raw_csv_data
import recipes_generator.model.recipe_model
from torch.optim import AdamW
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import random
import numpy as np

# Dataset creation

In [2]:
recipes1 = load_preprocess_raw_json_data('recipes_generator/data/recipes_raw_nosource_ar.json')
recipes2 = load_preprocess_raw_json_data('recipes_generator/data/recipes_raw_nosource_epi.json')
recipes3 = load_preprocess_raw_json_data('recipes_generator/data/recipes_raw_nosource_fn.json')

In [3]:
recipes4 = load_preprocess_raw_csv_data('recipes_generator/data/recipes_general.csv')

In [4]:
for recipe in recipes4[20:30]:
    print(recipe)

<|startoftext|>Prompt: beef eye round, dried thyme leaves, salt, pepper, ready-to-serve beef broth, Burgundy wine, garlic, cornstarch, frozen sugar snap peas
Title: Low-Fat Burgundy Beef & Vegetable Stew
Ingredients: 
- 1 1/2 tablespoon cornstarch
- 1 cup frozen sugar snap peas
- 1 glass Burgundy wine
- 1/2 lb beef eye round
- 1/2 ounce dried thyme leaves
- 1 cup ready-to-serve beef broth
- 1/2 clove garlic
- 3 pinches salt
- 5 1/2 pepper
Servings: 6
Instructions: Trim fat from beef cut into 1-inch pieces. In Dutch oven heat oil over medium high hunt until hot. Add beef (half at a time) and brown evenly stirring occasionally. Pour off drippings. Season with thyme salt and pepper. Stir in broth wine and garlic. Bring to boil; reduce heat to low. Cover tightly and simmer 1 1/2 hours. Add carrots and onions. Cover and continue cooking 35 to 40 minutes or until beef and vegetables are tender. Bring beef stew to a boil over medium-high heat. Add cornstarch mixture; cook and stir 1 minute. S

In [5]:
recipe_list = recipes1 + recipes2 + recipes3 + recipes4

In [6]:
print(len(recipes1))
print(len(recipes2))
print(len(recipes3))
print(len(recipes4))

39441
18145
55781
487639


In [7]:
random.shuffle(recipe_list)
train_list, test_list = recipe_list[:int(.8*len(recipe_list))], recipe_list[int(.8*len(recipe_list))]
print('Number of train data: ', len(train_list))
print('Number of test data: ', len(test_list))

Number of train data:  480804
Number of test data:  478


In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('distilbert/distilgpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
configuration = GPT2Config.from_pretrained('distilbert/distilgpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("distilbert/distilgpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [9]:
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

dataset = RecipeDataset(train_list, tokenizer)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
del dataset

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

batch_size = 8

432,723 training samples
48,081 validation samples


# Model training

In [10]:
# Finetune
import torch
import random

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


# some parameters I cooked up that work reasonably well

epochs = 2

# this produces sample output every 100 steps
sample_every = 1000
# I save the model every 5000 step
save_every = 5000
# save the model to this file name
save_file = 'recipes_generator/model/recipes_generation_model'



training_stats = []
print("Currently using device type: ", device)

model = model.to(device)


Currently using device type:  cuda


In [11]:
# Fine-tune the GPT-2 model on the recipe dataset
from recipes_generator.model.recipe_model import train_model
history = train_model(model, train_dataset, val_dataset, epochs, batch_size, device, save_file=save_file)

Total number of steps:  108180
Currently using device type:  cuda
======== Epoch 1 / 2 ========
Training...
Batch: 54088, Loss: 0.5402792096138: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54089/54089 [9:47:50<00:00,  1.53it/s]  Average training loss: 0.55
  Perplexity: 1.73
Running Validation...
Validation Loss: 0.4237533509731293: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6010/6010 [21:49<00:00,  4.59it/s]  Validation Loss: 0.48
  Validation perplexity: 1.61


======== Epoch 2 / 2 ========
Training...


Batch: 54089, Loss: 0.3778754770755768: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54090/54090 [9:48:50<00:00,  1.53it/s]



  Average training loss: 0.49
  Perplexity: 1.64

Running Validation...


Validation Loss: 0.5597129464149475: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6010/6010 [21:47<00:00,  4.60it/s]

  Validation Loss: 0.46
  Validation perplexity: 1.58

Training complete!


In [12]:
model.save_pretrained(save_file)
tokenizer.save_pretrained(save_file)

('recipes_generator/model/recipes_generation_model/tokenizer_config.json',
 'recipes_generator/model/recipes_generation_model/special_tokens_map.json',
 'recipes_generator/model/recipes_generation_model/vocab.json',
 'recipes_generator/model/recipes_generation_model/merges.txt',
 'recipes_generator/model/recipes_generation_model/added_tokens.json')

# Training results

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained(save_file, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained(save_file, output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained(save_file, config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50259, 768)

In [14]:
import pandas as pd
from recipes_generator.model.recipe_model import evaluate_model
test_dataset = RecipeDataset(test_list, tokenizer, max_length=768)
print('Testing...')
test_loss, test_perplexity = evaluate_model(model, test_dataset, batch_size, device)
test_eval_df = pd.DataFrame(columns = ["test_loss", "test_perplexity"])
test_eval_df['test_loss'] = test_loss
test_eval_df['test_perplexity'] = test_perplexity
test_eval_df.to_csv("recipes_generator/model/recipes_generation_model/test_eval.csv")

Testing...


Validation Loss: 19.67061424255371: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:14<00:00,  4.28it/s]

  Validation Loss: 19.35
  Validation perplexity: 254429424.00


In [ ]:
training_eval_df = pd.DataFrame(columns = ["epoch", "training loss", "validation loss", "train perplexity", "validation perplexity"])
train_loss = []
train_perp = []
valid_loss = []
valid_perp = []
for epoch in history:
    train_loss.append(epoch['Training Loss'])
    train_perp.append(epoch['Training Perplexity'])
    valid_loss.append(epoch['Valid. Loss'])
    valid_perp.append(epoch['Valid. Perplexity'])

training_eval_df['epoch'] = [x for x in range(len(training_stats))]
training_eval_df['training loss'] = train_loss
training_eval_df['validation loss'] = valid_loss
training_eval_df['train perplexity'] = train_perp
training_eval_df['validation perplexity'] = valid_perp

training_eval_df.to_csv("recipes_generator/model/recipes_generation_model/train_eval.csv")

# Examples of generated recipes

In [16]:
from recipes_generator.model.recipe_model import generate_recipe
prompts = ['chocolate,flour,sugar', 'chicken, flour, lemon, parmesan cheese', 'courgette, aubergine,flour,lemon', 'courgette, pepper, flour, mid-seasoned cheese']
generated_recipes = []
for p in prompts:    
    recipe = generate_recipe(p, model, tokenizer,device)
    print(recipe)
    generated_recipes.append(recipe)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: chocolate,flour,sugar, butter, sugar, eggs, flour, baking powder, salt
Title: Chocolate Cake
Ingredients: 
- 150 g flour
- 150 baking powder
- 150 g salt
- 150 eggs
- 150 g chocolate
- 2 cups sugar
- 1/2 ounce butter
Servings: 1 cake
Instructions: Preheat oven to 180°C. Grease and flour a 20cm round cake tin. Melt the chocolate in a heatproof bowl set over a pan of simmering water stirring occasionally until smooth. Set aside to cool slightly. Sift the flour baking powder and salt into a bowl. In a large bowl cream the butter and sugar until light and fluffy. Add the eggs one at a time beating well after each addition. Add the melted chocolate and mix well. Add the flour mixture and mix well. Pour the batter into the prepared cake tin and smooth the top. 
Bake for 40-45 minutes or until a skewer inserted into the cake comes out clean. Cool in the tin for 10 minutes then turn out onto a wire rack to cool completely.
Cook time: 45 min 
Preparation time: 20 min 
Total time: 1 h 5 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: chicken, flour, lemon, parmesan cheese, salt, pepper
Title: Chicken Parmesan
Ingredients: 
- 1 cup flour
- 1 pinch salt
- 1 pepper
- 1 lemon
- 1/2 cup parmesan cheese
- 1/2 chicken
Servings: 4
Instructions: Preheat oven to 350°F. Rinse chicken and pat dry with paper towels. Mix flour lemon zest lemon juice parmesan cheese salt and pepper in a plastic bag. Add chicken and shake to coat. Heat oil in a large skillet over medium-high heat. Add chicken and cook until browned on all sides about 5 minutes. Transfer chicken to a baking dish. Bake until chicken is cooked through about 30 minutes.
Cook time: 30 min 
Preparation time: 10 min 
Total time: 40 min 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: courgette, aubergine,flour,lemon, zest of, garlic clove, olive oil
Title: Courgette and Aubergine Gratin
Ingredients: 
- 1 garlic clove
- 1 aubergine
- 1 zest of
- 1 lb courgette
- 1 lemon
- 1 tablespoon olive oil
Servings: 4
Instructions: Preheat the oven to 400°F. Place the courgette and aubergine in a baking dish and season with salt and freshly ground black pepper. Sprinkle with the lemon zest and garlic and drizzle with the olive oil. Bake for 30 minutes or until the courgette and aubergine are tender.
Cook time: 30 min 
Preparation time: 10 min 
Total time: 40 min 
Prompt: courgette, pepper, flour, mid-seasoned cheese, olive oil, garlic clove
Title: Courgette Fritters
Ingredients: 
- 1 cup flour
- 1 garlic clove
- 1/2 pepper
- 1/2 lb courgette
- 1 tablespoon olive oil
- 1 cup mid-seasoned cheese
Servings: 4
Instructions: Cut courgette in half lengthwise and cut into thin slices. Mix flour and pepper in a bowl. Dip courgette slices in flour mixture. Heat oil in a large fry

In [17]:
prompts = ['chocolate,flour,sugar', 'chicken, flour, lemon, parmesan cheese', 'courgette, aubergine,flour,lemon', 'courgette, pepper, flour, mid-seasoned cheese']
model_loaded = GPT2LMHeadModel.from_pretrained(save_file, ignore_mismatched_sizes=True).to(device)
tokenizer_loaded = GPT2TokenizerFast.from_pretrained("tokenizer")
generated_recipes_from_loaded = []
for p in prompts:    
    recipe = generate_recipe(p, model_loaded, tokenizer_loaded,device)
    print(recipe)
    generated_recipes_from_loaded.append(recipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: chocolate,flour,sugar, unsalted butter, eggs, vanilla extract
Title: Chocolate Cake
Ingredients: 
- 150 g vanilla extract
- 150 g unsalted butter
- 150 eggs
- 3 cupsflour
- 2 cups sugar
- 2 lbs chocolate
Servings: 1
Instructions: Preheat oven to 180°C. Grease and line a 20cm springform tin with baking paper. Melt the chocolate in a heatproof bowl set over a pan of simmering water stirring until smooth. Set aside to cool slightly. Sift the flour and sugar together into a large bowl. Add the melted chocolate and stir to combine. Add the melted butter eggs and vanilla and stir to combine. Pour the mixture into the prepared tin and smooth the top. Bake for 1 hour or until a skewer inserted into the centre of the cake comes out clean. 
Cool in the tin for 10 minutes then turn out onto a wire rack to cool completely.
Cook time: 1 h 
Preparation time: 15 min 
Total time: 1 h 15 min 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: chicken, flour, lemon, parmesan cheese, salt, pepper, olive oil, butter, onion, garlic cloves, dry white wine, chicken broth, fresh parsley
Title: Chicken Piccata
Ingredients: 
- 1 cup flour
- 1 pinch salt
- 1/4 glass dry white wine
- 1/4 pepper
- 1/4 cup chicken broth
- 1/4 lb onion
- 1/4 garlic cloves
- 1/4 ounce butter
- 2 lemon
- 1 cup parmesan cheese
- 1 tablespoon olive oil
- 1/2 chicken
- 1 tablespoon fresh parsley
Servings: 4
Instructions: Sprinkle chicken with salt and pepper. Dredge chicken in flour shaking off excess. Heat oil and butter in a large skillet over medium-high heat. Add chicken and cook turning once until browned about 10 minutes. Transfer chicken to a plate. Add onion and garlic to skillet and cook stirring occasionally until softened about 5 minutes. Add wine and broth and bring to a boil scraping up browned bits. Return chicken to skillet. Cover reduce heat to medium-low and simmer until chicken is cooked through about 20 minutes. 
Transfer chicken to

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=768) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt: courgette, aubergine,flour,lemon, garlic clove, olive oil, parmesan cheese
Title: Courgette and aubergine
Ingredients: 
- 1 garlic clove
- 1 aubergine
- 1 lemon
- 1 cup flour
- 1 lb courgette
- 1 tablespoon olive oil
- 1 cup parmesan cheese
Servings: 4
Instructions: Peel the courgette and cut into thin slices. Peel the aubergine and cut into thin slices. Put the flour in a bowl and season with salt and pepper. Dredge the courgette slices in the flour shaking off any excess. Heat the oil in a large frying pan and add the courgette slices and aubergine slices. Cook over a medium heat for 3-4 minutes or until the aubergine is tender. Add the lemon juice and garlic and cook for a further 2-3 minutes or until the aubergine is tender. 
Sprinkle with the parmesan and serve immediately.
Cook time: 10 min 
Preparation time: 10 min 
Total time: 20 min 
Prompt: courgette, pepper, flour, mid-seasoned cheese, olive oil
Title: Courgette Fritters
Ingredients: 
- 1 cup flour
- 1 pepper
- 1/2 l